In [1]:
%%configure -f 
{
    "conf":
    {
        "spark.app.name":"Bhavesh_Relevancy_notebook",
        "spark.yarn.queue": "default",
        "spark.jars": "/apps/Jars/obelisk-retail-legos.jar,/apps/Jars/mysql-connector-java-5.1.40.jar",
        "spark.driver.extraJavaOptions": "-Dscala.color",
        "spark.driver.memory": "2g",
        "spark.executor.memory": "2g",
        "spark.executor.instances": "4"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1209,application_1685341929318_0823,spark,dead,Link,,
1214,application_1685341929318_0829,spark,dead,Link,,


In [2]:
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.Constants.STANDARD_COL_NAMES._
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.io._
import ai.couture.obelisk.commons.utils.DateTimeUtil._
import org.apache.spark.sql._
import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import spark.implicits._

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1215,application_1685341929318_0831,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.Constants.STANDARD_COL_NAMES._
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.io._
import ai.couture.obelisk.commons.utils.DateTimeUtil._
import org.apache.spark.sql._
import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import spark.implicits._


In [3]:
var getDaysInBetween: UserDefinedFunction = udf((startDate: String, endDate: String) => {
    getNumberOfDaysBetweenTwoDates(startDate, endDate, "yyyy-MM-dd")
})

def saveNLoadDF(key: String)(df: DataFrame): DataFrame = {
    var tempPath = "/data/Archive/bhavesh/inventoryPrediction/temp/exp_files"
    DFToParquet.putDF(tempPath + "/" + key, df)
    ParquetToDF.getDF(tempPath + "/" + key)
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

getDaysInBetween: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function2>,LongType,Some(List(StringType, StringType)))
saveNLoadDF: (key: String)(df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame


In [4]:
var interactionsDB = ParquetToDF.getDF("/data/ecomm/ajio/processed/interactionsDB")
interactionsDB = (
    interactionsDB.filter(col(DATE).between("2022-03-13", "2023-03-13") && col("event") === "Checkout" && col("purchase") === "New" 
                          && col(USERID).isNotNull && col(QUANTITY) > 0 && col("booked_rev") > 0 && col(PRODUCTID).isNotNull
                         )
    .groupBy(PRODUCTID, DATE)
    .agg(sum(QUANTITY) as "sales")
    .transform(saveNLoadDF("salesDayWise"))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

interactionsDB: org.apache.spark.sql.DataFrame = [productid: string, date: date ... 1 more field]


In [5]:
var distinctDates = interactionsDB.select(DATE).distinct()
var distinctProducts = interactionsDB.select(PRODUCTID).distinct()
var distinctCombinations = distinctProducts.crossJoin(broadcast(distinctDates))

var operationDF = (
    distinctCombinations.join(interactionsDB, Seq(PRODUCTID, DATE), "left")
    .na.fill(0)
    .withColumn("sales", col("sales").cast(DoubleType))
    .transform(saveNLoadDF("salesCompleteTrain"))
    .withColumn("daysInHistory", getDaysInBetween(col(DATE), lit("2023-03-15")) - lit(1))
    .withColumn("monthsInHistory", floor(col("daysInHistory")/30) + 1)
    .filter(col("monthsInHistory") <= 12)
    .repartition(210)
    .transform(saveNLoadDF("operationDF"))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

operationDF: org.apache.spark.sql.DataFrame = [productid: string, date: date ... 3 more fields]


In [6]:
var percentileLimits = operationDF.groupBy(PRODUCTID).agg(
    callUDF("percentile_approx", col("sales"), lit(0.1)).alias("percentile_10"),
    callUDF("percentile_approx", col("sales"), lit(0.2)).alias("percentile_20"),
    callUDF("percentile_approx", col("sales"), lit(0.3)).alias("percentile_30"),
    callUDF("percentile_approx", col("sales"), lit(0.4)).alias("percentile_40"),
    callUDF("percentile_approx", col("sales"), lit(0.5)).alias("percentile_50"),
    callUDF("percentile_approx", col("sales"), lit(0.6)).alias("percentile_60"),
    callUDF("percentile_approx", col("sales"), lit(0.7)).alias("percentile_70"),
    callUDF("percentile_approx", col("sales"), lit(0.8)).alias("percentile_80"),
    callUDF("percentile_approx", col("sales"), lit(0.9)).alias("percentile_90"),
    callUDF("percentile_approx", col("sales"), lit(1.0)).alias("percentile_100")
).transform(saveNLoadDF("productPercentiles"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from http://10.144.97.194:8999/sessions/1187/statements/5 with error payload: "requirement failed: Session isn't active."


In [ ]:
val binColumns = Array("percentile_10", "percentile_30", "percentile_50", "percentile_70", "percentile_90")

// Use the stack function to stack the columns into key-value pairs
val stackedDF = percentileLimits.select($"productid", expr(s"stack(${binColumns.length}, ${binColumns.map(c => s"'$c', $c").mkString(", ")})").as("bin", "value"))

var percentileLimitsBinned = stackedDF.withColumn("bin", substring(col("bin"), 12, 2).cast(IntegerType)).withColumn((col("bin") + lit(10))/lit(20)).transform(saveNLoadDF("percentileLimitsBinned"))

In [ ]:
    var X = operationDF.join(percentileLimits, Seq(PRODUCTID)).withColumn(
      "bin",
      when(col("sales") <= col("percentile_20"), lit(1)).otherwise(
        when(col("sales") <= col("percentile_40"), lit(2)).otherwise(
          when(col("sales") <= col("percentile_60"), lit(3)).otherwise(
            when(col("sales") <= col("percentile_80"), lit(4)).otherwise(
              lit(5)
            )
          )
        )
      )
    ).select(PRODUCTID, "monthsInHistory", "bin", "daysInHistory").transform(saveNLoadDF("operationDFBinned"))
X = X.groupBy(PRODUCTID, "monthsInHistory", "bin").agg(count("daysInHistory") as "count").groupBy(PRODUCTID, "bin").agg(avg("count") as "count").transform(saveNLoadDF("featureStats"))

In [ ]:
var y = (
    ParquetToDF.getDF("/data/ecomm/ajio/processed/interactionsDB")
    .filter(col(DATE).between("2023-03-15", "2023-04-14") && col("event") === "Checkout" && col("purchase") === "New" 
                          && col(USERID).isNotNull && col(QUANTITY) > 0 && col("booked_rev") > 0 && col(PRODUCTID).isNotNull
                         )
    .groupBy(PRODUCTID, DATE)
    .agg(sum(QUANTITY).cast(DoubleType) as "sales")
    .transform(saveNLoadDF("salesDayWiseTarget"))
)

var distinctDatesTarget = y.select(DATE).distinct()
var distinctProductsTarget = y.select(PRODUCTID).distinct()
var distinctCombinationsTarget = distinctProductsTarget.crossJoin(broadcast(distinctDatesTarget))
y = distinctCombinationsTarget.join(y, Seq(PRODUCTID, DATE), "left").na.fill(0).transform(saveNLoadDF("operationDFTarget"))
y = (
    y.join(percetileLimits, Seq(PRODUCTID))
    .withColumn("bin",
                when(col("sales")<=col("percentile_20", lit(1))).otherwise(
                    when(col("sales")<=col("percentile_40"), lit(2)).otherwise(
                        when(col("sales")<=col("percentile_60"), lit(3)).otherwise(
                            when("sales")<=col("percentile_80", lit(4)).otherwise(
                                lit(5)
                            )
                        )
                    )
                )
            )
    .select(PRODUCTID, "bin", DATE)
    .repartition(210)
    .transform(saveNLoadDF("operationDFTargetBinned"))
    .groupBy(PRODUCTID, "bin")
    .agg(count(DATE) as "count")
    .transform(saveNLoadDF("targetStats"))
)

In [ ]:
package ai.couture.obelisk.retail.legos.inventoryprediction

import ai.couture.obelisk.commons.utils.BaseBlocks
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.Constants.STANDARD_COL_NAMES._
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.io._
import ai.couture.obelisk.commons.utils.DateTimeUtil._
import org.apache.spark.sql._
import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

object PercentileBinnedComputation extends BaseBlocks {

  var getDaysInBetween: UserDefinedFunction = udf((startDate: String, endDate: String) => {
    getNumberOfDaysBetweenTwoDates(startDate, endDate, "yyyy-MM-dd")
  })

  override def load(): Unit = {

  }

  override def doTransformations(): Unit = {
      
      var interactionsDB = ParquetToDF.getDF("/data/ecomm/ajio/processed/interactionsDB")
    interactionsDB = (
        interactionsDB.filter(col(DATE).between("2022-03-13", "2023-03-13") && col("event") === "Checkout" && col("purchase") === "New" 
                              && col(USERID).isNotNull && col(QUANTITY) > 0 && col("booked_rev") > 0 && col(PRODUCTID).isNotNull
                             )
        .groupBy(PRODUCTID, DATE)
        .agg(sum(QUANTITY) as "sales")
        .transform(saveNLoadDF("salesDayWise"))
    )

    var trimmedProducts = ParquetToDF.getDF("/data/Archive/bhavesh/inventoryPrediction/TransformedDataset/date_when_prediction_is_made=2023-04-16/combined/data/test")
      .select(PRODUCTID).distinct()

    val distinctDates: DataFrame = interactionsDB.select(DATE).distinct()
    var operationDF = trimmedProducts.crossJoin(broadcast(distinctDates))
      .join(interactionsDB, Seq(PRODUCTID, DATE), "left")
      .na.fill(0)
      .withColumn("sales", col("sales").cast(DoubleType))
      .transform(saveNLoadDF("trimmed/salesCompleteTrain"))
      .withColumn("daysInHistory", getDaysInBetween(col(DATE), lit("2023-03-15")) - lit(1))
      .withColumn("monthsInHistory", floor(col("daysInHistory") / 30) + 1)
      .filter(col("monthsInHistory") <= 12)
      .transform(saveNLoadDF("trimmed/operationDF"))

    var percentileLimits = operationDF.groupBy(PRODUCTID).agg(
      callUDF("percentile_approx", col("sales"), lit(0.1)).alias("percentile_10"),
      callUDF("percentile_approx", col("sales"), lit(0.2)).alias("percentile_20"),
      callUDF("percentile_approx", col("sales"), lit(0.3)).alias("percentile_30"),
      callUDF("percentile_approx", col("sales"), lit(0.4)).alias("percentile_40"),
      callUDF("percentile_approx", col("sales"), lit(0.5)).alias("percentile_50"),
      callUDF("percentile_approx", col("sales"), lit(0.6)).alias("percentile_60"),
      callUDF("percentile_approx", col("sales"), lit(0.7)).alias("percentile_70"),
      callUDF("percentile_approx", col("sales"), lit(0.8)).alias("percentile_80"),
      callUDF("percentile_approx", col("sales"), lit(0.9)).alias("percentile_90"),
      callUDF("percentile_approx", col("sales"), lit(1.0)).alias("percentile_100")
    ).transform(saveNLoadDF("trimmed/productPercentiles"))

    var X = operationDF.join(percentileLimits, Seq(PRODUCTID)).withColumn(
      "bin",
      when(col("sales") <= col("percentile_20"), lit(1)).otherwise(
        when(col("sales") <= col("percentile_40"), lit(2)).otherwise(
          when(col("sales") <= col("percentile_60"), lit(3)).otherwise(
            when(col("sales") <= col("percentile_80"), lit(4)).otherwise(
              lit(5)
            )
          )
        )
      )
    ).select(PRODUCTID, "monthsInHistory", "bin", "daysInHistory")
      .transform(saveNLoadDF("trimmed/operationDFBinned"))
    X = X.groupBy(PRODUCTID, "monthsInHistory", "bin")
      .agg(count("daysInHistory") as "count")
      .groupBy(PRODUCTID, "bin")
      .agg(sum("count").cast(DoubleType) / lit(12.0) as "count")
      .transform(saveNLoadDF("trimmed/featureStats"))

    var y = (
      ParquetToDF.getDF("/data/ecomm/ajio/processed/interactionsDB")
        .filter(col(DATE).between("2023-03-15", "2023-04-14") && col("event") === "Checkout" && col("purchase") === "New"
          && col(USERID).isNotNull && col(QUANTITY) > 0 && col("booked_rev") > 0 && col(PRODUCTID).isNotNull
        )
        .groupBy(PRODUCTID, DATE)
        .agg(sum(QUANTITY).cast(DoubleType) as "sales")
        .transform(saveNLoadDF("salesDayWiseTarget"))
      )

    var distinctDatesTarget = y.select(DATE).distinct()
    var distinctCombinationsTarget = trimmedProducts.crossJoin(broadcast(distinctDatesTarget))
    y = distinctCombinationsTarget.join(y, Seq(PRODUCTID, DATE), "left")
      .na.fill(0)
      .transform(saveNLoadDF("trimmed/operationDFTarget"))
    y = (
      y.join(percentileLimits, Seq(PRODUCTID))
        .withColumn("bin",
          when(col("sales") <= col("percentile_20"), lit(1)).otherwise(
            when(col("sales") <= col("percentile_40"), lit(2)).otherwise(
              when(col("sales") <= col("percentile_60"), lit(3)).otherwise(
                when(col("sales") <= col("percentile_80"), lit(4)).otherwise(
                  lit(5)
                )
              )
            )
          )
        )
        .select(PRODUCTID, "bin", DATE)
        .repartition(210)
        .transform(saveNLoadDF("trimmed/operationDFTargetBinned"))
        .groupBy(PRODUCTID, "bin")
        .agg(count(DATE) as "count")
        .transform(saveNLoadDF("trimmed/targetStats"))
      )

    val binColumns = Array("percentile_10", "percentile_30", "percentile_50", "percentile_70", "percentile_90")

    // Use the stack function to stack the columns into key-value pairs
    val stackedDF = percentileLimits.select(col("productid"),
      expr(s"stack(${binColumns.length}, ${binColumns.map(c => s"'$c', $c").mkString(", ")}) as (bin, value)"))

    var percentileLimitsBinned = stackedDF.withColumn("bin", substring(col("bin"), 12, 2).cast(IntegerType))
      .withColumn("bin", (col("bin") + lit(10)) / lit(20))
      .transform(saveNLoadDF("trimmed/percentileLimitsBinned"))
      
    var test = X.toDF(PRODUCTID, "bin", "count_x").join(y.toDF(PRODUCTID, "bin", "count_y"), Seq(PRODUCTID, "bin"), "outer").na.fill(0.0).withColumn("count", (col("count_x") + col("count_y"))/lit(2.0)).drop("count_x", "count_y")
    test = test.transform(saveNLoadDF("trimmed/test"))
    test = test.join(percentileLimitsBinned, Seq(PRODUCTID, "bin")).withColumn("predictedyQuantity", col("count") * col("value")).groupBy(PRODUCTID).agg(sum("predictedyQuantity") as "predictedyQuantity").transform(saveNLoadDF("trimmed/predictions"))
      
      // Later split predictions into menShirts/womenKurtas according to productAttrs
  }

  def saveNLoadDF(key: String)(df: DataFrame): DataFrame = {
    var tempPath = "/data/Archive/bhavesh/inventoryPrediction/temp/exp_files"
    DFToParquet.putDF(tempPath + "/" + key, df)
    ParquetToDF.getDF(tempPath + "/" + key)
  }

  override def save(): Unit = {

  }
}

In [4]:
var x = ParquetToDF.getDF("/data/Archive/bhavesh/inventoryPrediction/temp/exp_files/trimmed/featureStats").withColumn("count", col("count").cast(DoubleType))
var y = ParquetToDF.getDF("/data/Archive/bhavesh/inventoryPrediction/temp/exp_files/trimmed/targetStats").withColumn("count", col("count").cast(DoubleType))
x.printSchema
y.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

x: org.apache.spark.sql.DataFrame = [productid: string, bin: int ... 1 more field]
y: org.apache.spark.sql.DataFrame = [productid: string, bin: int ... 1 more field]
root
 |-- productid: string (nullable = true)
 |-- bin: integer (nullable = true)
 |-- count: double (nullable = true)

root
 |-- productid: string (nullable = true)
 |-- bin: integer (nullable = true)
 |-- count: double (nullable = true)



In [5]:
var test = x.toDF(PRODUCTID, "bin", "count_x").join(y.toDF(PRODUCTID, "bin", "count_y"), Seq(PRODUCTID, "bin"), "outer").na.fill(0.0).withColumn("count", (col("count_x") + col("count_y"))/lit(2.0)).drop("count_x", "count_y")
test = test.transform(saveNLoadDF("trimmed/test"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

test: org.apache.spark.sql.DataFrame = [productid: string, bin: int ... 1 more field]
test: org.apache.spark.sql.DataFrame = [productid: string, bin: int ... 1 more field]


In [6]:
test.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- productid: string (nullable = true)
 |-- bin: integer (nullable = true)
 |-- count: double (nullable = true)



In [7]:
var percentileLimitsBinned = ParquetToDF.getDF("/data/Archive/bhavesh/inventoryPrediction/temp/exp_files/trimmed/percentileLimitsBinned")
percentileLimitsBinned.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

percentileLimitsBinned: org.apache.spark.sql.DataFrame = [productid: string, bin: double ... 1 more field]
root
 |-- productid: string (nullable = true)
 |-- bin: double (nullable = true)
 |-- value: double (nullable = true)



In [8]:
test = test.join(percentileLimitsBinned, Seq(PRODUCTID, "bin")).withColumn("predictedyQuantity", col("count") * col("value")).groupBy(PRODUCTID).agg(sum("predictedyQuantity") as "predictedyQuantity").transform(saveNLoadDF("trimmed/predictions"))
test.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

test: org.apache.spark.sql.DataFrame = [productid: string, predictedyQuantity: double]
root
 |-- productid: string (nullable = true)
 |-- predictedyQuantity: double (nullable = true)



In [9]:
var productAttrs = ParquetToDF.getDF("/data/Archive/inventory/productAttributesLegosFNL").select(PRODUCTID, SIMILAR_GROUP_LEVEL)
productAttrs.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productAttrs: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string]
root
 |-- productid: string (nullable = true)
 |-- similargrouplevel: string (nullable = true)



In [10]:
test = test.join(productAttrs, Seq(PRODUCTID))
DFToParquet.putDF("/data/Archive/bhavesh/inventoryPrediction/experiments/exp_my_exp/predictions/ModelForEachBrickProductLevel/menShirts/predictions", test.filter(col(SIMILAR_GROUP_LEVEL) === "830216013").drop(SIMILAR_GROUP_LEVEL))
DFToParquet.putDF("/data/Archive/bhavesh/inventoryPrediction/experiments/exp_my_exp/predictions/ModelForEachBrickProductLevel/womenKurtas/predictions", test.filter(col(SIMILAR_GROUP_LEVEL) === "830303011").drop(SIMILAR_GROUP_LEVEL))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

test: org.apache.spark.sql.DataFrame = [productid: string, predictedyQuantity: double ... 1 more field]


In [20]:
def thresholdColumns(columns: Array[String], minThreshold: Double = 0, maxThreshold: Double = Double.PositiveInfinity)(df: DataFrame): DataFrame = {

    var scaledDF = df
    columns.foreach(
    column => {
    scaledDF = scaledDF.withColumn(column,
      when(col(column) > maxThreshold, lit(maxThreshold)).otherwise(col(column)))
      .withColumn(column, when(col(column) < minThreshold, lit(minThreshold)).otherwise(col(column)))
    })
    scaledDF
}

var suffix: String = "colorfamily_pricebucket_styletype_pattern_sleeve_brandname"
var prodLevelSigma = ParquetToDF.getDF("/data/Archive/bhavesh/inventoryPrediction/quantityStatsAcrossMonths/date_when_prediction_is_made=2023-04-16/productLevel/past12MonthsStats").select(PRODUCTID, "stddev")
var queryLevelSigma = ParquetToDF.getDF(s"/data/Archive/bhavesh/inventoryPrediction/quantityStatsAcrossMonths/date_when_prediction_is_made=2023-04-16/queryLevel/suffix=$suffix/past12MonthsStats").select("query", "stddev", SIMILAR_GROUP_LEVEL)
var productQueryMap = ParquetToDF.getDF(s"/data/Archive/bhavesh/inventoryPrediction/queryToProductMap/date_when_prediction_is_made=2023-04-16/suffix=$suffix")

def convertLocalToRequiredFormat(dateForWhichPredictionsAreMade: String, basePath: String, level: String, localFile: String, fileType: String = "csv"): Unit = {
    
    var actual = ParquetToDF.getDF(s"/data/Archive/bhavesh/inventoryPrediction/actualDataForPredictionPeriod/date=$dateForWhichPredictionsAreMade/sales")
    var fileName: String = if(level == "product"){
       "productLevelPredictions"
    } else{
        "queryAggregatedPredictions"
    }
    def doConversion(brick: String, brickId: String): DataFrame = {
        var localPredictionsPath = s"$basePath/$brick/$localFile"
        var df = if(fileType=="csv"){
            CSVToDF.getDF(localPredictionsPath, inferSchema = true)
        }
        else{
            ParquetToDF.getDF(localPredictionsPath)
        }

        
        df = df
        .withColumn(SIMILAR_GROUP_LEVEL, lit(brickId))
        .withColumn("model", lit("lr"))
        .withColumn("predicted", col("predictedyQuantity"))
        .select(PRODUCTID, SIMILAR_GROUP_LEVEL, "predicted", "model")
        
        if(level == "product") {
            df.transform(thresholdColumns(Array("predicted")))
            .join(prodLevelSigma, Seq(PRODUCTID))
            .withColumn("lowerBound", col("predicted") - (lit(2.0) * col("stddev")))
            .withColumn("upperBound", col("predicted") + (lit(2.0) * col("stddev")))
            .transform(thresholdColumns(Array("lowerBound")))
        }
        else {
            df.transform(thresholdColumns(Array("predicted")))
            .join(productQueryMap, Seq(SIMILAR_GROUP_LEVEL, PRODUCTID))
            .groupBy("model", "query", SIMILAR_GROUP_LEVEL)
            .agg(sum("predicted") as "predicted")
            .join(queryLevelSigma, Seq(SIMILAR_GROUP_LEVEL, "query"))
            .withColumn("lowerBound", col("predicted") - (lit(2.0) * col("stddev")))
            .withColumn("upperBound", col("predicted") + (lit(2.0) * col("stddev")))
            .transform(thresholdColumns(Array("lowerBound")))
        }
    }
    var menShirts = doConversion("menShirts", "830216013")
    DFToParquet.putDF(s"$basePath/menShirts/$fileName", menShirts, partitionedColumn="model")
    var womenKurtas = doConversion("womenKurtas", "830303011")
    DFToParquet.putDF(s"$basePath/womenKurtas/$fileName", womenKurtas, partitionedColumn="model")
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

thresholdColumns: (columns: Array[String], minThreshold: Double, maxThreshold: Double)(df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame
suffix: String = colorfamily_pricebucket_styletype_pattern_sleeve_brandname
prodLevelSigma: org.apache.spark.sql.DataFrame = [productid: string, stddev: double]
queryLevelSigma: org.apache.spark.sql.DataFrame = [query: string, stddev: double ... 1 more field]
productQueryMap: org.apache.spark.sql.DataFrame = [query: string, productid: string ... 2 more fields]
convertLocalToRequiredFormat: (dateForWhichPredictionsAreMade: String, basePath: String, level: String, localFile: String, fileType: String)Unit


In [12]:
def convertLocal(exp: String, date: String, filename: String, format: String){
    convertLocalToRequiredFormat(date, s"/data/Archive/bhavesh/inventoryPrediction/experiments/exp_$exp/predictions/ModelForEachBrickProductLevel", "product", filename, format)
    convertLocalToRequiredFormat(date, s"/data/Archive/bhavesh/inventoryPrediction/experiments/exp_$exp/predictions/ModelForEachBrickProductLevel", "query", filename, format)
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

convertLocal: (exp: String, date: String, filename: String, format: String)Unit


In [13]:
convertLocal(
    exp = "my_exp",
    date = "2023-04-16",
    filename = "predictions",
    format = "parquet"
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Combine arima and ml predictions

In [62]:
var ml_predictions_ms = ParquetToDF.getDF("/data/Archive/bhavesh/inventoryPrediction/experiments/exp_my_exp/predictions/ModelForEachBrickProductLevel/menShirts/queryAggregatedPredictions/model=lr")
var ml_predictions_wk = ParquetToDF.getDF("/data/Archive/bhavesh/inventoryPrediction/experiments/exp_my_exp/predictions/ModelForEachBrickProductLevel/womenKurtas/queryAggregatedPredictions/model=lr")

ml_predictions_ms.printSchema
ml_predictions_wk.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ml_predictions_ms: org.apache.spark.sql.DataFrame = [similargrouplevel: string, query: string ... 4 more fields]
ml_predictions_wk: org.apache.spark.sql.DataFrame = [similargrouplevel: string, query: string ... 4 more fields]
root
 |-- similargrouplevel: string (nullable = true)
 |-- query: string (nullable = true)
 |-- predicted: double (nullable = true)
 |-- stddev: double (nullable = true)
 |-- lowerBound: double (nullable = true)
 |-- upperBound: double (nullable = true)

root
 |-- similargrouplevel: string (nullable = true)
 |-- query: string (nullable = true)
 |-- predicted: double (nullable = true)
 |-- stddev: double (nullable = true)
 |-- lowerBound: double (nullable = true)
 |-- upperBound: double (nullable = true)



In [63]:
var ml_predictions = ml_predictions_ms.select(SIMILAR_GROUP_LEVEL, "query", "predicted").union(ml_predictions_wk.select(SIMILAR_GROUP_LEVEL, "query", "predicted"))
ml_predictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ml_predictions: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [similargrouplevel: string, query: string ... 1 more field]
root
 |-- similargrouplevel: string (nullable = true)
 |-- query: string (nullable = true)
 |-- predicted: double (nullable = true)



In [64]:
DFToParquet.putDF("/data/Archive/bhavesh/inventoryPrediction/temp/ml_predictions", ml_predictions)
ml_predictions = ParquetToDF.getDF("/data/Archive/bhavesh/inventoryPrediction/temp/ml_predictions")
ml_predictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ml_predictions: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [similargrouplevel: string, query: string ... 1 more field]
root
 |-- similargrouplevel: string (nullable = true)
 |-- query: string (nullable = true)
 |-- predicted: double (nullable = true)



In [21]:
var baseline = ParquetToDF.getDF("/data/Archive/inventory/2023-04-15/queryProductMap_colorfamily_pricebucket_styletype_pattern_sleeve_brandname/past1MonthHistorical/queryPredictions")
baseline.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

baseline: org.apache.spark.sql.DataFrame = [query: string, predicted: double ... 4 more fields]
root
 |-- query: string (nullable = true)
 |-- predicted: double (nullable = true)
 |-- stddev: double (nullable = true)
 |-- lowerBound: double (nullable = true)
 |-- upperBound: double (nullable = true)
 |-- similargrouplevel: integer (nullable = true)



In [22]:
baseline = baseline.select(SIMILAR_GROUP_LEVEL, "query", "predicted")
baseline.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

baseline: org.apache.spark.sql.DataFrame = [similargrouplevel: int, query: string ... 1 more field]
root
 |-- similargrouplevel: integer (nullable = true)
 |-- query: string (nullable = true)
 |-- predicted: double (nullable = true)



In [23]:
var arima_predictions = ParquetToDF.getDF("/data/Archive/inventory/2023-04-15/queryProductMap_colorfamily_pricebucket_styletype_pattern_sleeve_brandname/autoarima/queryPredictions")
arima_predictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

arima_predictions: org.apache.spark.sql.DataFrame = [query: string, predicted: double ... 5 more fields]
root
 |-- query: string (nullable = true)
 |-- predicted: double (nullable = true)
 |-- stddev: double (nullable = true)
 |-- avg: double (nullable = true)
 |-- lowerBound: double (nullable = true)
 |-- upperBound: double (nullable = true)
 |-- similargrouplevel: integer (nullable = true)



In [24]:
arima_predictions = arima_predictions.select(SIMILAR_GROUP_LEVEL, "query", "predicted")
arima_predictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

arima_predictions: org.apache.spark.sql.DataFrame = [similargrouplevel: int, query: string ... 1 more field]
root
 |-- similargrouplevel: integer (nullable = true)
 |-- query: string (nullable = true)
 |-- predicted: double (nullable = true)



In [67]:
println(ml_predictions.count, arima_predictions.count)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(377851,318156)


In [25]:
println(baseline.count, arima_predictions.count)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(337326,318156)


In [68]:
ml_predictions = ml_predictions.withColumn("model", lit("ml"))
arima_predictions = arima_predictions.withColumn("model", lit("arima"))

var predictions = ml_predictions.select(arima_predictions.columns.map(col): _*).union(arima_predictions)
predictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ml_predictions: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [similargrouplevel: string, query: string ... 2 more fields]
arima_predictions: org.apache.spark.sql.DataFrame = [similargrouplevel: int, query: string ... 2 more fields]
predictions: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [similargrouplevel: string, query: string ... 2 more fields]
root
 |-- similargrouplevel: string (nullable = true)
 |-- query: string (nullable = true)
 |-- predicted: double (nullable = true)
 |-- model: string (nullable = false)



In [26]:
baseline = baseline.withColumn("model", lit("baseline"))
arima_predictions = arima_predictions.withColumn("model", lit("arima"))

var predictions = baseline.select(arima_predictions.columns.map(col): _*).union(arima_predictions)
predictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

baseline: org.apache.spark.sql.DataFrame = [similargrouplevel: int, query: string ... 2 more fields]
arima_predictions: org.apache.spark.sql.DataFrame = [similargrouplevel: int, query: string ... 2 more fields]
predictions: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [similargrouplevel: int, query: string ... 2 more fields]
root
 |-- similargrouplevel: integer (nullable = true)
 |-- query: string (nullable = true)
 |-- predicted: double (nullable = true)
 |-- model: string (nullable = false)



In [27]:
predictions = predictions.groupBy(SIMILAR_GROUP_LEVEL, "query").agg(avg("predicted") as "predicted")
predictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

predictions: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [similargrouplevel: int, query: string ... 1 more field]
root
 |-- similargrouplevel: integer (nullable = true)
 |-- query: string (nullable = true)
 |-- predicted: double (nullable = true)



In [28]:
DFToParquet.putDF("/data/Archive/bhavesh/inventoryPrediction/experiments/exp_combined_arima_baseline/combinedPredictions2", predictions)
predictions = ParquetToDF.getDF("/data/Archive/bhavesh/inventoryPrediction/experiments/exp_combined_arima_baseline/combinedPredictions2")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

predictions: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [similargrouplevel: int, query: string ... 1 more field]


In [29]:
predictions.groupBy(SIMILAR_GROUP_LEVEL).agg(countDistinct("query")).show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+---------------------+
|similargrouplevel|count(DISTINCT query)|
+-----------------+---------------------+
|830216013        |227431               |
|830303011        |142073               |
+-----------------+---------------------+



In [30]:
predictions.count

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res29: Long = 369504


In [31]:
var predictions = ParquetToDF.getDF("/data/Archive/bhavesh/inventoryPrediction/experiments/exp_combined_arima_baseline/combinedPredictions2")
predictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

predictions: org.apache.spark.sql.DataFrame = [similargrouplevel: int, query: string ... 1 more field]
root
 |-- similargrouplevel: integer (nullable = true)
 |-- query: string (nullable = true)
 |-- predicted: double (nullable = true)



In [32]:
predictions = (
    predictions.join(queryLevelSigma, Seq(SIMILAR_GROUP_LEVEL, "query"))
    .withColumn("lowerBound", col("predicted") - (lit(2.0) * col("stddev")))
    .withColumn("upperBound", col("predicted") + (lit(2.0) * col("stddev")))
    .transform(thresholdColumns(Array("lowerBound")))
)
predictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

predictions: org.apache.spark.sql.DataFrame = [similargrouplevel: int, query: string ... 4 more fields]
root
 |-- similargrouplevel: integer (nullable = true)
 |-- query: string (nullable = true)
 |-- predicted: double (nullable = true)
 |-- stddev: double (nullable = true)
 |-- lowerBound: double (nullable = true)
 |-- upperBound: double (nullable = true)



In [33]:
DFToParquet.putDF("/data/Archive/bhavesh/inventoryPrediction/experiments/exp_combined_arima_baseline/combinedPredictions", predictions, partitionedColumn="similargrouplevel")
HdfsUtils.delete("/data/Archive/bhavesh/inventoryPrediction/experiments/exp_combined_arima_baseline/combinedPredictions2")
predictions = ParquetToDF.getDF("/data/Archive/bhavesh/inventoryPrediction/experiments/exp_combined_arima_baseline/combinedPredictions")
predictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

predictions: org.apache.spark.sql.DataFrame = [query: string, predicted: double ... 4 more fields]
root
 |-- query: string (nullable = true)
 |-- predicted: double (nullable = true)
 |-- stddev: double (nullable = true)
 |-- lowerBound: double (nullable = true)
 |-- upperBound: double (nullable = true)
 |-- similargrouplevel: integer (nullable = true)

